In [1]:
import numpy as np
import pandas as pd

In [2]:
import re
import gensim
from gensim.models.word2vec import Word2Vec

In [3]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [4]:
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
import xgboost as xgb

In [5]:
from keras.models import Sequential
from keras.layers import Dense

In [6]:
data = pd.read_csv('/content/tweets.csv')

In [7]:
pd.set_option('display.max_colwidth' , None)

In [8]:
data

,id,label,tweet
0,1,0,#fingerprint #Pregnancy Test https://goo.gl/h1MfQV #android #apps #beautiful #cute #health #igers #iphoneonly #iphonesia #iphone
1,2,0,Finally a transparant silicon case ^^ Thanks to my uncle :) #yay #Sony #Xperia #S #sonyexperias… http://instagram.com/p/YGEt5JC6JM/
2,3,0,We love this! Would you go? #talk #makememories #unplug #relax #iphone #smartphone #wifi #connect... http://fb.me/6N3LsUpCu
3,4,0,I'm wired I know I'm George I was made that way ;) #iphone #cute #daventry #home http://instagr.am/p/Li_5_ujS4k/
4,5,1,What amazing service! Apple won't even talk to me about a question I have unless I pay them $19.95 for their stupid support!
...,...,...,...
7915,7916,0,Live out loud #lol #liveoutloud #selfie #smile #sony #music #headphones https://instagram.com/p/5spiNsJ_c9/
7916,7917,0,"We would like to wish you an amazing day! Make every minute count #tls #today #iphone #accessories #news #life February 23, 2017 at 0…"
7917,7918,0,Helping my lovely 90 year old neighbor with her iPad this morning has just made me realise that 'I' don't actually need an I pad!
7918,7919,0,"Finally got my #smart #pocket #wifi stay connected anytime,anywhere! #ipad and #samsung #s3 #gadget # http://instagr.am/p/U-53G_vJU8/"


In [9]:
data.drop('id',axis=1,inplace=True)

In [10]:
data.head()

,label,tweet
0,0,#fingerprint #Pregnancy Test https://goo.gl/h1MfQV #android #apps #beautiful #cute #health #igers #iphoneonly #iphonesia #iphone
1,0,Finally a transparant silicon case ^^ Thanks to my uncle :) #yay #Sony #Xperia #S #sonyexperias… http://instagram.com/p/YGEt5JC6JM/
2,0,We love this! Would you go? #talk #makememories #unplug #relax #iphone #smartphone #wifi #connect... http://fb.me/6N3LsUpCu
3,0,I'm wired I know I'm George I was made that way ;) #iphone #cute #daventry #home http://instagr.am/p/Li_5_ujS4k/
4,1,What amazing service! Apple won't even talk to me about a question I have unless I pay them $19.95 for their stupid support!


In [11]:
data.isna().sum()

label    0
tweet    0
dtype: int64

In [12]:
data['label'].value_counts()

0    5894
1    2026
Name: label, dtype: int64

In [13]:
data.head()

,label,tweet
0,0,#fingerprint #Pregnancy Test https://goo.gl/h1MfQV #android #apps #beautiful #cute #health #igers #iphoneonly #iphonesia #iphone
1,0,Finally a transparant silicon case ^^ Thanks to my uncle :) #yay #Sony #Xperia #S #sonyexperias… http://instagram.com/p/YGEt5JC6JM/
2,0,We love this! Would you go? #talk #makememories #unplug #relax #iphone #smartphone #wifi #connect... http://fb.me/6N3LsUpCu
3,0,I'm wired I know I'm George I was made that way ;) #iphone #cute #daventry #home http://instagr.am/p/Li_5_ujS4k/
4,1,What amazing service! Apple won't even talk to me about a question I have unless I pay them $19.95 for their stupid support!


In [14]:
#url removing

In [15]:
def remove_URL(sample):
  return re.sub(r"http\S+", "", sample)

In [16]:
data['text_clean_url-less'] = data['tweet'].apply(lambda x: remove_URL(x))

In [17]:
#simple preprocessing using gensim

In [18]:
data['clean_tweet'] = data['text_clean_url-less'].apply(lambda x: gensim.utils.simple_preprocess(x))

In [19]:
data.head()

,label,tweet,text_clean_url-less,clean_tweet
0,0,#fingerprint #Pregnancy Test https://goo.gl/h1MfQV #android #apps #beautiful #cute #health #igers #iphoneonly #iphonesia #iphone,#fingerprint #Pregnancy Test #android #apps #beautiful #cute #health #igers #iphoneonly #iphonesia #iphone,"[fingerprint, pregnancy, test, android, apps, beautiful, cute, health, igers, iphoneonly, iphonesia, iphone]"
1,0,Finally a transparant silicon case ^^ Thanks to my uncle :) #yay #Sony #Xperia #S #sonyexperias… http://instagram.com/p/YGEt5JC6JM/,Finally a transparant silicon case ^^ Thanks to my uncle :) #yay #Sony #Xperia #S #sonyexperias…,"[finally, transparant, silicon, case, thanks, to, my, uncle, yay, sony, xperia, sonyexperias]"
2,0,We love this! Would you go? #talk #makememories #unplug #relax #iphone #smartphone #wifi #connect... http://fb.me/6N3LsUpCu,We love this! Would you go? #talk #makememories #unplug #relax #iphone #smartphone #wifi #connect...,"[we, love, this, would, you, go, talk, makememories, unplug, relax, iphone, smartphone, wifi, connect]"
3,0,I'm wired I know I'm George I was made that way ;) #iphone #cute #daventry #home http://instagr.am/p/Li_5_ujS4k/,I'm wired I know I'm George I was made that way ;) #iphone #cute #daventry #home,"[wired, know, george, was, made, that, way, iphone, cute, daventry, home]"
4,1,What amazing service! Apple won't even talk to me about a question I have unless I pay them $19.95 for their stupid support!,What amazing service! Apple won't even talk to me about a question I have unless I pay them $19.95 for their stupid support!,"[what, amazing, service, apple, won, even, talk, to, me, about, question, have, unless, pay, them, for, their, stupid, support]"


In [20]:
data.drop(['tweet','text_clean_url-less'],axis = 1 ,inplace=True)

In [21]:
data.head()

,label,clean_tweet
0,0,"[fingerprint, pregnancy, test, android, apps, beautiful, cute, health, igers, iphoneonly, iphonesia, iphone]"
1,0,"[finally, transparant, silicon, case, thanks, to, my, uncle, yay, sony, xperia, sonyexperias]"
2,0,"[we, love, this, would, you, go, talk, makememories, unplug, relax, iphone, smartphone, wifi, connect]"
3,0,"[wired, know, george, was, made, that, way, iphone, cute, daventry, home]"
4,1,"[what, amazing, service, apple, won, even, talk, to, me, about, question, have, unless, pay, them, for, their, stupid, support]"


In [22]:
data.shape

(7920, 2)

In [23]:
#splitting data into train data and test data

In [24]:
x_train,x_test,y_train,y_test = train_test_split(data['clean_tweet'],data['label'],test_size=0.2,random_state=42)

In [25]:
#word embedding with word2vec

In [26]:
model_gs = Word2Vec(x_train, min_count = 1)

In [27]:
model_gs.wv.most_similar('apple')

[('mac', 0.9995951652526855),
 ('macbook', 0.9995089769363403),
 ('fuckyou', 0.9994006752967834),
 ('os', 0.9989251494407654),
 ('job', 0.998862087726593),
 ('first', 0.998815655708313),
 ('tomorrow', 0.9987751841545105),
 ('here', 0.9987040758132935),
 ('away', 0.998690128326416),
 ('upgrade', 0.9986879229545593)]

In [28]:
words = model_gs.wv.index_to_key

In [29]:
len(words)

14946

In [30]:
x_train_vec = np.array([np.array([model_gs.wv[i] for i in ls if i in words]) for ls in x_train],dtype=object)
x_test_vec = np.array([np.array([model_gs.wv[i] for i in ls if i in words]) for ls in x_test],dtype=object)

In [31]:
len(x_train_vec[0][0])


100

In [32]:
x_train_vec.ndim

1

In [33]:
x_train_vec_avg = []
for v in x_train_vec:
  if v.size:
    x_train_vec_avg.append(v.mean(axis = 0))
  else:
    x_train_vec_avg.append(np.zeros(100,dtype=float))
x_test_vec_avg = []
for v in x_test_vec:
  if v.size:
    x_test_vec_avg.append(v.mean(axis = 0))
  else:
    x_test_vec_avg.append(np.zeros(100,dtype=float))

Trying out different models

In [34]:
#Random Forest Classifier

In [35]:
clf = RandomForestClassifier(n_estimators=100)
clf.fit(x_train_vec_avg,y_train)

RandomForestClassifier()

In [36]:
#training accuracy
y_pred = clf.predict(x_train_vec_avg)
accuracy_score(y_pred,y_train)

0.9995265151515151

In [37]:
#testing

In [38]:
y_pred = clf.predict(x_test_vec_avg)
accuracy_score(y_pred,y_test)

0.8428030303030303

In [39]:
#SVC

In [40]:
svc = SVC(kernel='rbf')

In [41]:
svc.fit(x_train_vec_avg,y_train)

SVC()

In [42]:
#training accuracy svm
y_pred = svc.predict(x_train_vec_avg)
accuracy_score(y_pred,y_train)

0.842645202020202

In [43]:
#testing svc

In [44]:
y_pred = svc.predict(x_test_vec_avg)
accuracy_score(y_pred,y_test)

0.8428030303030303

In [45]:
#XGB

In [46]:
xgb_clf = xgb.XGBClassifier(n_estimators=20 ,max_depth = 1, random_state = 42, learning_rate = 0.4, gamma = 0.1 )

In [47]:
xgb_clf.fit(x_train_vec_avg,y_train)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=0.1, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.4, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=1, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=20, n_jobs=None,
              num_parallel_tree=None, random_state=42, ...)

In [48]:
#training accuracy xgb
y_pred = xgb_clf.predict(x_train_vec_avg)
accuracy_score(y_pred,y_train)

0.8478535353535354

In [49]:
#testing xgb

In [50]:
y_pred = xgb_clf.predict(x_test_vec_avg)
accuracy_score(y_pred,y_test)

0.8459595959595959

In [51]:
#KNN

In [52]:
knn = KNeighborsClassifier(n_neighbors = 25, metric = 'minkowski',p = 1)

In [53]:
knn.fit(x_train_vec_avg,y_train)

KNeighborsClassifier(n_neighbors=25, p=1)

In [54]:
#training accuracy knn
y_pred = knn.predict(x_train_vec_avg)
accuracy_score(y_pred,y_train)

0.8500631313131313

In [55]:
y_pred = knn.predict(x_test_vec_avg)
accuracy_score(y_pred,y_test)

0.8465909090909091

In [90]:
#Neural network

In [109]:
model = Sequential()

In [117]:
model.add(Dense(85,activation = 'relu'))
model.add(Dense(80,activation = 'relu'))
model.add(Dense(65,activation = 'relu'))
model.add(Dense(1, activation = 'sigmoid'))

In [118]:
model.compile(loss = 'binary_crossentropy',optimizer = 'adam',metrics = ['accuracy'])

In [119]:
x_train_new = np.array(x_train_vec_avg)
y_train_new = np.array(y_train)

In [120]:
history = model.fit(x_train_new,y_train_new,epochs=100,validation_split=0.2)

Epoch 1/100
159/159 [==============================] - 2s 5ms/step - loss: 0.5587 - accuracy: 0.7498 - val_loss: 0.4367 - val_accuracy: 0.7429
Epoch 2/100
159/159 [==============================] - 1s 3ms/step - loss: 0.3569 - accuracy: 0.8315 - val_loss: 0.3471 - val_accuracy: 0.8573
Epoch 3/100
159/159 [==============================] - 1s 4ms/step - loss: 0.3441 - accuracy: 0.8425 - val_loss: 0.3346 - val_accuracy: 0.8580
Epoch 4/100
159/159 [==============================] - 1s 4ms/step - loss: 0.3400 - accuracy: 0.8396 - val_loss: 0.3489 - val_accuracy: 0.8383
Epoch 5/100
159/159 [==============================] - 1s 4ms/step - loss: 0.3381 - accuracy: 0.8459 - val_loss: 0.3289 - val_accuracy: 0.8596
Epoch 6/100
159/159 [==============================] - 1s 3ms/step - loss: 0.3353 - accuracy: 0.8435 - val_loss: 0.3279 - val_accuracy: 0.8636
Epoch 7/100
159/159 [==============================] - 1s 3ms/step - loss: 0.3311 - accuracy: 0.8467 - val_loss: 0.3353 - val_accuracy: 0.8462

In [121]:
x_test_new = np.array(x_test_vec_avg)
y_test_new = np.array(y_test)

In [122]:
#testing

In [123]:
model.evaluate(x_test_new,y_test_new)

50/50 [==============================] - 0s 2ms/step - loss: 0.3297 - accuracy: 0.8510


[0.32972267270088196, 0.8510100841522217]

Picking the better performing model

In [124]:
#Neural network was the better performing model

In [125]:
#random tweet test

In [126]:
label_dict = {1 :'Negative sentiment on the product',0 :'Positive sentiment on the product'}

In [127]:
def sentiment_analyze(tweet):
  tweet = remove_URL(tweet)
  tweet1 = gensim.utils.simple_preprocess(tweet)
  tweet_vec_avg = np.array([model_gs.wv[i] for i in tweet1 if i in words]).mean(axis = 0)
  test = tweet_vec_avg.reshape(1,100)
  sentiment = model.predict(test)
  sentiment[sentiment<0.5] = 0
  sentiment[sentiment>0.5] = 1
  print(label_dict[sentiment[0][0]])

In [128]:
tweet = 'wow!!! my apple iphone 11 is simply the best #iphonelife #love #awesome'

In [129]:
sentiment_analyze(tweet)

1/1 [==============================] - 0s 81ms/step
Positive sentiment on the product


In [130]:
tweet2 = 'wow!! apple devices have become a nightmare.stupid phone'

In [131]:
sentiment_analyze(tweet2)

1/1 [==============================] - 0s 23ms/step
Negative sentiment on the product
